In [47]:
import pandas as pd
import numpy as np
import time
import requests
import re

In [48]:
# Helper functions
def timer_function(func=None, n_run=1):

    ti = time.time()
    for n in range(n_run):
        func
    tf = time.time()
    
    print(f'Runtime: {(tf - ti)/n_run} s')

def read_input_pd(file_path=None):
    df_data = pd.read_csv(file_path, sep=',', header=None)
    
    return df_data

def read_input_lst(file_path=None):
    with open(file_path,'r') as f:
        lines = f.read().splitlines()

    input_arr = list(lines)


    return input_arr

def scrape_data(day):
    """
    fetches the input data from the website
    day: the day of december to fetch
    """
    # Fetch session cookie value from local file
    with open("session.txt",'r') as f:
        key = f.read()
    cookie = {'session': key}
    url= f"https://adventofcode.com/2021/day/{day}/input"

    with requests.Session() as s:
        r = s.get(url, cookies = cookie).text
        # Data always end with a blanc line. Data is split on line change
        input_lst = re.split('\n', r)[:-1]

    return input_lst

# December 1


In [78]:
# data import
input_test = np.array([199,200,208,210,200,207,240,269,260,263])
input_1 = scrape_data(day=1)
input_1 = [int(i) for i in input_1]
print('Number of input rows:', len(input_1))

Number of input rows: 2000


In [79]:
# Part 1
def dec1_part1(data):
    answer = np.sum(np.diff(data) > 0)
    return answer

answer = dec1_part1(input_1)
print(f'Solution december 1. - part 1: ', answer)
timer_function(func=dec1_part1(input_1), n_run=1000000)

Solution december 1. - part 1:  1583
Runtime: 1.3995647430419921e-08 s


In [80]:
# Part 2
def dec1_part2(data, window):
    rolling_sum =[np.sum(data[i:i + window]) for i in range(0, len(data) + 1 - window)]
    answer = dec1_part1(rolling_sum)

    return rolling_sum, answer

rolling_sum, answer = dec1_part2(input_1, window=3)
print(f'Solution december 1. - part 2: ', answer)
timer_function(func=dec1_part2(input_1, window=3), n_run=1000000)

Solution december 1. - part 2:  1627
Runtime: 1.4998435974121095e-08 s


# December 2


In [67]:
input_test = ['forward 5','down 5','forward 8','up 3','down 8','forward 2']
input_2 = scrape_data(day=2)
print('Number of input rows:', len(input_2))

Number of input rows: 1000


In [69]:
# Solution and test input
def split_direction(df, direction, sign):
    mask = df['raw'].str.contains(direction)
    df.loc[mask, 'direction'] = direction[0]
    df.loc[mask, 'step'] = df.loc[mask, 'raw'].str.replace(direction, '').astype(int)*sign
    
    return df

def dec2_part1(data):
    df_data = pd.DataFrame(data).rename(columns={0:'raw'})
    
    df_data = split_direction(df_data, 'forward ', +1)
    df_data = split_direction(df_data, 'down ', +1)
    df_data = split_direction(df_data, 'up ', -1)
    
    mask_fwd = df_data['direction'] == 'f'
    pos_horizontal = df_data.loc[mask_fwd, 'step'].sum()
    pos_vertical = df_data.loc[~mask_fwd, 'step'].sum()
    
    answer = pos_horizontal*pos_vertical
    
    return pos_horizontal, pos_vertical, df_data, answer


pos_horizontal, pos_vertical, df_input, answer = dec2_part1(input_2)
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution: ', answer)
timer_function(func=dec2_part1(input_2), n_run=100000)

Position (horizontal/vertical): 1939.0 , 1109.0
Solution:  2150351.0
Runtime: 1.9991397857666014e-08 s


In [71]:
# December  2 - Part 2
def dec2_part2(df):    
    mask_fwd = df['direction'] == 'f'
    
    df['aim_step'] = 0
    df.loc[~mask_fwd, 'aim_step'] = df.loc[~mask_fwd, 'step']
    df['aim'] = df['aim_step'].cumsum()

    pos_horizontal = df.loc[mask_fwd, 'step'].sum()
    pos_vertical = (df.loc[mask_fwd, 'step']*df.loc[mask_fwd, 'aim']).sum()
    
    answer = pos_horizontal*pos_vertical
    
    return pos_horizontal, pos_vertical, df, answer

pos_horizontal, pos_vertical, df_input, answer = dec2_part2(df_input)
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution: ', answer)
timer_function(func=dec2_part2(df_input), n_run=10000000)

Position (horizontal/vertical): 1939.0 , 950357.0
Solution:  1842742223.0
Runtime: 1.5710687637329103e-08 s


# December 3

In [73]:
input_test = ['00100','11110','10110','10111','10101','01111','00111','11100','10000','11001','00010','01010']
input_3 = scrape_data(day=3)
print('Number of input rows:', len(input_3))

Number of input rows: 1000


In [74]:
# Solution part 1 and test
def dec3_part1(data):
    n_digit = len(data[0])
    n_row = len(data)
    
    digit_sum = np.zeros(n_digit)
    
    for row in data:
        row_lst = [int(i) for i in row]
        digit_sum = np.add(digit_sum, row_lst)
    
    mask = digit_sum >= n_row/2
    gamma = ''.join(map(str, 1*(mask)))
    epsilon = ''.join(map(str, 1*(~mask)))
    
    answer = int(gamma,2)*int(epsilon,2)
    
    return gamma, epsilon, answer
    
print(dec3_part1(input_3))
timer_function(func=dec3_part1(input_3), n_run=1000000)

('010001110111', '101110001000', 3374136)
Runtime: 1.4997005462646484e-08 s


In [75]:
def find_rating(rating, bit):
    n_digit = len(rating[0])
    n_row = len(rating)
    idx = 0
    
    while (len(rating) > 1) & (idx < n_digit):
        digit_sum = 0
        for row in rating:
            digit_sum += int(row[idx])
            
        digit_even = digit_sum == len(rating)/2
        
        if digit_even:
            rating = [entry for entry in rating if entry[idx] == bit]
        else:
            if bit == '0':
                digit_match = 1*(digit_sum < len(rating)/2)
            else:
                digit_match = 1*(digit_sum > len(rating)/2)

            rating = [entry for entry in rating if entry[idx] == str(digit_match)]

        idx += 1
    
    # select the only remaining correct entry if more exist
    rating = [entry for entry in rating if entry[-1] == bit][0]
    
    return rating

def dec3_part2(data):
    o_rating = find_rating(rating=data, bit='1')
    co2_rating = find_rating(rating=data, bit='0')
        
    answer = int(o_rating,2)*int(co2_rating,2)
    
    return o_rating, co2_rating, answer

print(dec3_part2(input_3))
timer_function(func=dec3_part2(input_3), n_run=1000000)

('011101110101', '100100010010', 4432698)
Runtime: 1.5999317169189453e-08 s
